#Step 1: install all require packages

In [ ]:
!pip install -q accelerate peft bitsandbytes transformers trl

In [ ]:
!huggingface-cli login --token ****

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `learning` has been saved to /home/bach/.cache/huggingface/stored_tokens
Your token has been saved to /home/bach/.cache/huggingface/token
Login successful.
The current active token is: `learning`


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/home/bach/anaconda3/envs/finetune/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#**In case of Llama 3.2, the following prompt template is used for the chat models**


<|start_header_id|>system<|end_header_id|>

{System_messages}<|eot_id|>

<|start_header_id|>user<|end_header_id|>

{user_messages}<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>

{assistant_messages}<|eot_id|>

In [1]:
# <|start_header_id|>system<|end_header_id|>
# You are an expert in composing functions. You are given a question and a set of possible functions. 
# Based on the question, you will need to make one or more function/tool calls to achieve the purpose. 
# If none of the functions can be used, point it out. If the given question lacks the parameters required by the function,also point it out. You should only return the function call in tools call sections.
# If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]
# You SHOULD NOT include any other text in the response.
# Here is a list of functions in JSON format that you can invoke.[
#     {
#         "name": "get_user_info",
#         "description": "Retrieve details for a specific user by their unique identifier. Note that the provided function is in Python 3 syntax.",
#         "parameters": {
#             "type": "dict",
#             "required": [
#                 "user_id"
#             ],
#             "properties": {
#                 "user_id": {
#                 "type": "integer",
#                 "description": "The unique identifier of the user. It is used to fetch the specific user details from the database."
#             },
#             "special": {
#                 "type": "string",
#                 "description": "Any special information or parameters that need to be considered while fetching user details.",
#                 "default": "none"
#                 }
#             }
#         }
#     }
# ]
# <|eot_id|><|start_header_id|>user<|end_header_id|>

# Can you retrieve the details for the user with the ID 7890, who has black as their special request?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

- Orignal Dataset: https://huggingface.co/datasets/timdettmers/openassistant-guanaco

In [ ]:
#use to reformat dataset to suitable form with llama 3.2
def transform_conversations(example):
    text_conversation = example["text"]
    segments = text_conversation.split("###")
    reformatted_segments = []
    for i in range(1,len(segments)-1,2):       
        human_text = segments[i].replace('Human:', '').strip()
        if(i+1 < len(segments)):
            assistant_text = segments[i+1].replace('Assistant:', '').strip()
        # Apply the new template
            reformatted_segments.append(f'<|start_header_id|>user<|end_header_id|> {human_text} <|eot_id|> <|start_header_id|>assistant<|end_header_id|> {assistant_text} <|eot_id|>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<|start_header_id|>user<|end_header_id|> {human_text} <|eot_id|>')
    return "".join(reformatted_segments)

#Step3: Load model and config

QLoRA will use a rank of 64 with a scaling parameter of 16. We’ll load the Llama 2 model directly in 4-bit precision using the NF4 type and train it for one epoch

In [8]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-3.2-1B"

#transform dataset to suitable prompt used in llama 3.2
dataset_name = "timdettmers/openassistant-guanaco"

# Fine-tuned model name
new_model = "meta-llama/Llama-3.2-1B-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

#**Step 4:Load everything and start the fine-tuning process**

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")


# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    gradient_checkpointing = True
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    formatting_func=transform_conversations,
    processing_class=tokenizer,
    args=training_arguments
)

# Train model
trainer.train()

In [ ]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
#garbage trash
import gc
gc.collect()

#**Step 7: Store New Llama2 Model (Llama-2-7b-chat-finetune)**

How can we store our new Llama-2-7b-chat-finetune model now? We need to merge the weights from LoRA with the base model. Unfortunately, as far as I know, there is no straightforward way to do it: we need to reload the base model in FP16 precision and use the peft library to merge everything.

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"